In [1]:
import org.apache.spark.sql.SparkSession

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.0.191:4040
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1575249527713)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession


In [2]:
object SparkSessionCreator {

  def sparkSessionCreate(): SparkSession = {

    SparkSession
      .builder()
      .master("local[*]")
      .appName("Hedge-fund-application-realtime-analysis")
      .getOrCreate()

  }

}

defined object SparkSessionCreator


In [3]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions.lit

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions.lit


In [4]:
object DataSourcer {

  def rawTrainData(sparkSession: SparkSession): DataFrame = {
         
      // load train data from local
        sparkSession.read.option("header", "true").csv("./data/train.csv")
    }

  def rawTestData(sparkSession: SparkSession): DataFrame = {

    // load train data from local
    // populate label col - not included in raw test data
    sparkSession.read.option("header", "true").csv("./data/test.csv")
      .withColumn("Open", lit("0"))

  }

}

defined object DataSourcer


In [5]:
object DataCleaner {

  // function to produce a clean data frame from a raw data frame
  def cleanData(dataFrame: DataFrame): DataFrame = {

    // def function to format data correctly
    def formatData(dataFrame: DataFrame): DataFrame = {

      dataFrame
        .withColumn("Open", dataFrame("Open").cast("Double"))
        .withColumn("High", dataFrame("High").cast("Double"))
        .withColumn("Low", dataFrame("Low").cast("Double"))
        .withColumn("Close", dataFrame("Close").cast("Double"))
        .withColumn("Volume", dataFrame("Volume").cast("Double"))
        .withColumnRenamed("Open", "label")

    }

    

    // format raw data
    val outputData = formatData(dataFrame)


    // return cleaned data frame
    outputData

  }

}

defined object DataCleaner


In [33]:
object OutputSaver {

  // function to save a fitted pipeline
  def pipelineSaver(pipelineModel: PipelineModel): Unit = {

    pipelineModel
      .write
      .overwrite()
      .save("./pipelines/fitted-pipeline")

  }

  // function to save predictions
//   def predictionsSaver(dataFrame: DataFrame): Unit = {

//     dataFrame
//       .select("PassengerId", "prediction")
//       .withColumnRenamed("prediction", "Survived")
//       .write
//       .mode(saveMode = SaveMode.Overwrite)
//       .csv(path = "./predictions/predictions_csv")

//   }

}

defined object OutputSaver


In [31]:
import org.apache.spark.ml.{Pipeline, PipelineModel}

import org.apache.spark.ml.{Pipeline, PipelineModel}


In [38]:
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.regression.GeneralizedLinearRegression
import org.apache.spark.ml.regression.{RandomForestRegressionModel, RandomForestRegressor}
import org.apache.spark.ml.regression.{GBTRegressionModel, GBTRegressor}
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorAssembler, VectorIndexer}

object MachineLearning {

  def pipelineFit(dataFrame: DataFrame): Unit = {
      
    // define feature vector assembler
    val featureAssembler = new VectorAssembler()
      .setInputCols(Array[String](
          "High",
          "Low",
          "Close",
          "Volume"
        )
      )
      .setOutputCol("features")

   val Array(trainingData, testData) = dataFrame.randomSplit(Array(0.7, 0.3))  
      
    var df_train = featureAssembler.transform(trainingData)
    var df_test =  featureAssembler.transform(testData) 
      
//     //LINEAR REGRESSION  
      
//     //Model Creation  
//     val lr = new LinearRegression()
  
//     // Fit the model
//     val lrModel = lr.fit(df)
      
    
//     //GeneralizedLinearRegression
      
//     //Model Creation
//     val glr = new GeneralizedLinearRegression()
//       .setFamily("gaussian")
//       .setLink("identity")
//       .setMaxIter(10)
//       .setRegParam(0.3)
      
//     // Fit the Model
//     val glrModel = glr.fit(df)
      
//         val featureIndexer = new VectorIndexer()
//           .setInputCol("features")
//           .setOutputCol("label")
//           .setMaxCategories(4)
//           .fit(data)
      
//       val rf = new RandomForestRegressor()
//           .setLabelCol("label")
//           .setFeaturesCol("features")
      
//       val rfrModel = rf.fit(df_train)
//       val predictions = rfrModel.transform(df_test)
//       predictions.select("prediction", "label", "features").show(5)
      
//       // Select (prediction, true label) and compute test error.
//         val evaluator = new RegressionEvaluator()
//           .setLabelCol("label")
//           .setPredictionCol("prediction")
//           .setMetricName("rmse")
//         val rmse = evaluator.evaluate(predictions)
//         println(s"Root Mean Squared Error (RMSE) on test data = $rmse")
      
      
      //////////////////////////////////////////////////////////////////////////////////////
      
      val gbt = new GBTRegressor()
              .setLabelCol("label")
              .setFeaturesCol("features")
              .setMaxIter(10)
      
      val gbtModel = rf.fit(df_train)
      val predictions = rfrModel.transform(df_test)
      predictions.select("prediction", "label", "features").show(5)
      
      // Select (prediction, true label) and compute test error.
        val evaluator = new RegressionEvaluator()
          .setLabelCol("label")
          .setPredictionCol("prediction")
          .setMetricName("rmse")
        val rmse = evaluator.evaluate(predictions)
        println(s"Root Mean Squared Error (RMSE) on test data = $rmse")
      
      ///////////////////
      
      // chain indexers and forest in a Pipeline
            val pipeline = new Pipeline()
              .setStages(
                Array(
                  featureAssembler,
                  gbt
                )
              )
      
          val pipelineModel_gbt = pipeline.fit(dataFrame)

          OutputSaver.pipelineSaver(pipelineModel = pipelineModel_gbt)
      
      
        }

}
    

<console>: 139: error: not found: value rf

In [41]:
    //LINEAR REGRESSION  
    println("Linear Regression")
    println(s"Coefficients: ${lrModel.coefficients} Intercept: ${lrModel.intercept}")

    // Summarize the model over the training set and print out some metrics
    val trainingSummary_lr = lrModel.summary
    println(s"numIterations: ${trainingSummary_lr.totalIterations}")
    println(s"objectiveHistory: [${trainingSummary_lr.objectiveHistory.mkString(",")}]")
    trainingSummary_lr.residuals.show()
    println(s"RMSE: ${trainingSummary_lr.rootMeanSquaredError}")
    println(s"r2: ${trainingSummary_lr.r2}")
      
    println()
    println()  
      
   ////////////////////////////////////////////////////////////////////////////////////////
    
//     println(" Generalized Linear Regression")
//     println(s"Coefficients: ${glrModel.coefficients} Intercept: ${glrModel.intercept}")

//     // Summarize the model over the training set and print out some metrics
//     val trainingSummary_glr = glrModel.summary
//     //println(s"numIterations: ${trainingSummary_glr.totalIterations}")
//     println(s"objectiveHistory: [${trainingSummary_glr.objectiveHistory.mkString(",")}]")
//     trainingSummary_glr.residuals.show()
//     println(s"RMSE: ${trainingSummary_glr.rootMeanSquaredError}")
//     println(s"r2: ${trainingSummary_glr.r2}")
      
//     println()
//     println()     
      
      

  }

}

import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.regression.GeneralizedLinearRegression
import org.apache.spark.ml.feature.{IndexToString, StringIndexer, VectorAssembler, VectorIndexer}
defined object MachineLearning


In [7]:
    // create spark session
    val spark = SparkSessionCreator.sparkSessionCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@64e30ad0


In [8]:
    // train data
    val rawTrainData = DataSourcer.rawTrainData(sparkSession = spark)

rawTrainData: org.apache.spark.sql.DataFrame = [Open: string, High: string ... 3 more fields]


In [9]:
    // clean train data
    val cleanTrainData = DataCleaner.cleanData(dataFrame = rawTrainData)

cleanTrainData: org.apache.spark.sql.DataFrame = [label: double, High: double ... 3 more fields]


In [10]:
cleanTrainData.show()

+---------+---------+---------+---------+----------+
|    label|     High|      Low|    Close|    Volume|
+---------+---------+---------+---------+----------+
|28.459999|28.994286|28.338572|28.655714| 7.38143E7|
|    28.73|28.811428|28.395714|28.558571|1.062145E8|
|28.891428|28.967142|28.118572|28.138571|1.164408E8|
|28.422857|28.774286|27.964285|28.032858| 1.78815E8|
|28.202858|28.425714|28.038572|28.068571|1.121799E8|
|28.528572|28.554285|27.182858|27.617144|2.067212E8|
|27.617144|27.681429|26.954287|26.992857|1.786897E8|
|27.051428|27.478571|26.957144|27.124287|1.725997E8|
|27.325714|28.308571|27.187143|28.257143|1.711955E8|
|     28.5|28.528572|28.017143|28.061428|1.224174E8|
|28.254286|28.285715|27.632856|27.809999|1.074437E8|
|    27.91|28.204287|27.508572|28.139999|1.239476E8|
|27.975714|28.215714|27.610001|27.738571|1.048649E8|
|27.871429|28.071428|27.792856|27.861429| 8.82462E7|
|27.751429|27.857143|27.285715|27.408571| 9.72097E7|
|27.595715|27.928572|27.514286|27.918571|1.521

In [37]:
    // fitted pipeline
MachineLearning.pipelineFit(dataFrame = cleanTrainData)

+-----------------+---------+--------------------+
|       prediction|    label|            features|
+-----------------+---------+--------------------+
|28.46461044543444|27.871429|[28.071428,27.792...|
|28.46461044543444|27.984285|[28.085714,27.751...|
|28.46461044543444|28.059999|[28.214285,27.821...|
|28.46461044543444|28.202858|[28.425714,28.038...|
|28.46461044543444|28.254286|[28.285715,27.632...|
+-----------------+---------+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) on test data = 1.1965864737939682
